In [1]:
import numpy.linalg as npl
import numpy as np
import matplotlib.pyplot as plt
from timeit import Timer
from sympy import *
init_printing()

In [2]:
from Methods.methods import jacobi, sor, gauss_seidel, gauss, comp_w, error
from Helper.help_functions import characteristics, isDDM,  runntime_compare_df, print_lgs
from Helper.create_lgs import create_block_triag, band_matrix, create_upper_triangular

ModuleNotFoundError: No module named 'help_functions'

In [ ]:
mat, jac_it, gasei_it, sor_it, gael_it = [],[],[],[],[]
jac_time, gasei_time, sor_time_arr, gael_time = [],[],[],[]
jac_error, gasei_error, sor_error, gael_error = [],[],[],[]

normen = [None, 'inf', 0, 1, 2]  # 2-Norm, Maximumsnorm, sum(x!=0), 1-Norm

max_iter = 10000000
range = [2,4,5,10,20]

In [ ]:
for n in range:
    A,b  = create_block_triag(n)

    #A, b = band_matrix(n=n)
    #A,b=create_mat(n=n)
    print(f"Run: {n}")
    w =comp_w(A) # 1.25
    true_sol = npl.solve(A, b)
    jac_erg, k_jacobi = jacobi(A,b, iter=max_iter)
    t_j = Timer(lambda:jacobi(A,b, iter=max_iter))
    gauss_seidel_erg, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
    gs_t = Timer(lambda:gauss_seidel(A,b, iter=max_iter))
    sor_erg, k_sor = sor(A, b,w, iter=max_iter)
    sor_t = Timer(lambda:sor(A,b,w, iter=max_iter))
    gauss_elim_erg, c = gauss(A=A,B=b)
    t_ge = Timer(lambda:gauss(A,b))
    # iteration
    mat.append(n)
    jac_it.append(k_jacobi)
    gasei_it.append(k_gauss_seidel)
    sor_it.append(k_sor)
    gael_it.append(c)
    # error
    jac_error.append(error(jac_erg, true_sol))
    gasei_error.append(error(gauss_seidel_erg, true_sol))
    sor_error.append(error(sor_erg, true_sol))
    gael_error.append(error(gauss_elim_erg, true_sol))
    # timer
    jac_time.append("{:.3E}".format((t_j.timeit(1))))
    gasei_time.append("{:.3E}".format((gs_t.timeit(1))))
    sor_time_arr.append("{:.3E}".format((sor_t.timeit(1))))
    gael_time.append("{:.3E}".format((t_ge.timeit(1))))

In [ ]:
df = runntime_compare_df(mat, jac_it, gasei_it, sor_it, gael_it,jac_time, gasei_time, sor_time_arr, gael_time)

In [ ]:
df

In [ ]:
df_error = runntime_compare_df(mat, jac_it, gasei_it, sor_it, gael_it,
                               jac_time=jac_error, gasei_time=gasei_error, sor_time_arr=sor_error, gael_time=gael_error, label = "Fehler")

In [ ]:
df_error

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(mat, [float(e)for e in jac_time], label= "JAC",  linewidth=3)
plt.plot(mat, [float(e)  for e in gasei_time], label= "GS",  linewidth=3)
plt.plot( mat, [float(e) for e in sor_time_arr], label= "SOR",  linewidth=3,c="green")
plt.plot( mat, [float(e) for e in gael_time],  label= "Gauß Elimination",  linewidth=3,c="red")

plt.title('Laufzeitvergleich', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Laufzeit (s)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(mat, [float(e)for e in jac_error], label= "JAC",  linewidth=3)
plt.plot(mat, [float(e)  for e in gasei_error], label= "GS",  linewidth=3)
plt.plot( mat, [float(e) for e in sor_error], label= "SOR",  linewidth=3,c="green")
plt.plot( mat, [float(e) for e in gael_error],  label= "Gauß Elimination",  linewidth=3, c="red")

plt.title('Fehlervergleich', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Fehler (10E-6)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
w= 40
plt.bar(mat,jac_it, label= "JAC",width = w)
plt.bar(mat, gasei_it, label= "GS",width = w)
plt.bar( mat, sor_it, label= "SOR",width = w)
plt.bar( mat, gael_it,  label= "Gauß Elimination", width = w , color="red")

plt.title('Vergleich Iterationen', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Iterationen', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.show()

In [ ]:
df_error.plot.bar(figsize=(10,6))
plt.title('Vergleich Iterationen', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Iterationen (10E6)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='best', shadow=True, fontsize="large")
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
n=20
A = create_block_triag(n)
b = np.ones((n))
true_sol = npl.solve(A, b)

jac_erg, k_jacobi = jacobi(A,b, iter=max_iter)
gauss_seidel_erg, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
sor_erg, sor_time, k_sor = sor(A, b,w=1, iter=max_iter)
gauss_elim_erg,c = gauss(A,b)

In [ ]:
gs_fehler=np.round(gauss_seidel_erg[len(gauss_seidel_erg)-1]-true_sol,4)
jk_fehler=np.round(jac_erg[len(jac_erg)-1]-true_sol,4)
sor_fehler=np.round(sor_erg[len(sor_erg)-1]-true_sol,4)

jr = Matrix(jk_fehler)
gr = Matrix(gs_fehler)
sr = Matrix(sor_fehler)
#display(Math('\ \\text{ Jacobi : }%s   \\text{Gauss-Seidel :} %s \\text{SOR :} %s' % (sym.latex(sym.simplify(jr)) , (sym.latex(sym.simplify(gr))), (sym.latex(sym.simplify(sr)))  ))   )

In [ ]:
dict = {'Jacobi Verfahren': jac_erg,
        'Gauss-Seidel Verfahren': gauss_seidel_erg,
        "SOR Verfahren":sor_erg
        }
#label = [f'x{i}' for i in range(0,n)]
label = [f'x0 - x{n-1} ']

for verfahren_name,data in dict.items():

    plt.figure(figsize=(9,6))
    plt.plot(data) # linestyle='',marker='.'
    #plt.axvline(x=k_sor+0.5,linestyle='--',c='red')
    plt.title(verfahren_name, fontweight='bold', fontsize=17, pad=20)
    plt.xticks(rotation='0', fontsize=14)
    plt.yticks(rotation='0', fontsize=14)
    plt.xlabel('Iteration', fontsize=14, labelpad=15, weight ='bold')
    plt.ylabel('Wert', fontsize=14, labelpad=15, weight ='bold')
    plt.legend(labels = label,frameon=True,loc='best', shadow=True, fontsize="large")
    plt.tight_layout(h_pad=3.0)
    plt.grid(True)
    plt.show()

Ideen: LGS von n = 2 bis n = 6 mio

Plot: Fehler, Wert,